In [16]:
import pandas as pd
import numpy as np
import os
root_pth = "/home/user/data/THU-timeseries"
all_data_pth = [
    "MFRED/MFRED.csv",
    # "ETT-small/ETTh1.csv",
    # "ETT-small/ETTh2.csv",
    # "ETT-small/ETTm1.csv",
    # "ETT-small/ETTm2.csv",
    # 'exchange_rate/exchange_rate.csv',
    # 'illness/national_illness.csv',
    # 'weather/weather.csv',
]
for data_pth in all_data_pth:
    df = pd.read_csv(os.path.join(root_pth, data_pth), index_col=0, parse_dates=True)
    long_df = []
    for c in df.columns.tolist():
        temp_df = df[[c]].copy()
        temp_df['unique_id'] = c
        temp_df = temp_df.reset_index()
        temp_df = temp_df.rename(columns={c:'y','date':'ds'})
        if data_pth.__contains__('ETTh'):
            num_train = 12 * 30 * 24
            num_test = 4 * 30 * 24
            num_vali = 4 * 30 * 24
            temp_df = temp_df[:num_train+num_vali+num_test]
        elif data_pth.__contains__('ETTm'):
            num_train = 12 * 30 * 24 * 4
            num_test = 4 * 30 * 24 * 4
            num_vali = 4 * 30 * 24 * 4
            temp_df = temp_df[:num_train+num_vali+num_test]
        long_df.append(temp_df)
    long_df = pd.concat(long_df)
    long_df = long_df.reset_index(drop=True)
    long_df.to_csv(os.path.join(root_pth, data_pth[:-4]+'_NF.csv'), index=False)
    # break

In [2]:
import pandas as pd
import numpy as np
import os

In [21]:
# pth = "/home/user/data/Benchmarks/metric_results/long_term_forecast_ETTh1_96_192_DLinear_ETTh1_ftS_sl96_ll48_pl192_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0/true.npy"
pth = "/home/user/data/Benchmarks/metric_results/long_term_forecast_electricity_96_192_DLinear_custom_ftS_sl96_ll48_pl192_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0/true.npy"
y_true = np.load(pth)
y_true[0][:10]

array([[-0.61717033],
       [-0.78310746],
       [-0.90936404],
       [-1.1366258 ],
       [-1.205165  ],
       [-1.210576  ],
       [-1.3260106 ],
       [-1.4721075 ],
       [-1.63083   ],
       [-1.4522672 ]], dtype=float32)

In [24]:
import pandas as pd
from datasetsforecast.long_horizon import LongHorizon, LongHorizonInfo
import logging
import os
import argparse
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from neuralforecast import NeuralForecast
from neuralforecast.models import (
    NHITS,
    DLinear,
    PatchTST,
    TimesNet,
    # iTransformer,
    # TFT,
    Autoformer,
    # FEDformer,
    # LSTM,
    # MLP,
    # NBEATSx,
    # DeepAR,
)
from neuralforecast.losses.numpy import mse, mae
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss
from neuralforecast.losses.numpy import mqloss, mse, mae
from src.utils.metrics import calculate_metrics, get_bench_metrics
import pickle

# Change this to your own data to try the model
print(LongHorizonInfo['ECL'].test_size)
Y_df, _, _ = LongHorizon.load(directory='/home/user/data/NF_longterm', group='ECL')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

# For this excercise we are going to take 960 timestamps as validation and test
n_time = len(Y_df.ds.unique())
num_train = int(n_time * 0.7)
num_test = int(n_time * 0.2)
print(num_test)
num_vali = n_time - num_train - num_test
# num_train = 12 * 30 * 24
# num_test = 4 * 30 * 24
# num_vali = 4 * 30 * 24
        
# val_size = 96*10
# test_size = 96*10

Y_df = Y_df[Y_df['unique_id'] == 'OT']


5260
5260


In [15]:
level = [i for i in range(10, 100, 10)]
quantiles = [0.05 * (1 + i) for i in range(19)]
seq_len = 96
pred_len = 192
config = {
    "h": pred_len,
    "input_size": seq_len,
    "max_steps": 100,
    "loss": MQLoss(level=level),
    "early_stop_patience_steps": 5,
    "val_check_steps": 1,
    "batch_size": 32,
    "valid_batch_size":32,
    "inference_windows_batch_size": 32,
    "windows_batch_size": 32,
    "scaler_type": "standard",
    'devices':1,
    "fast_dev_run": False,
    'drop_last_loader': True,
    "num_sanity_val_steps": 0,
    "random_seed": 1,
    "enable_progress_bar": True,
    "accelerator":'cpu',
    "default_root_dir": "/home/user/data/FrequencyDiffusion/savings/mfred/benchmarks",
}

models = [
    # TSMixer(**config, n_series=n_series, loss=MQLoss(level=level)),
    # PatchTST(**config),
    # TimesNet(**config),
    DLinear(**config),
    # NHITS(**config),
    # Autoformer(**config)
    # FEDformer(**config, loss=MQLoss(level=level)),
    # DeepAR(
    #     **config,
    #     loss=DistributionLoss(distribution="Normal", num_samples=200, level=level),
    # ),
]
nf = NeuralForecast(models=models, freq="5min")


Y_hat_df = nf.cross_validation(
    df=Y_df,
    val_size=num_vali,
    test_size=num_test,
    n_windows=None,
)
# cols = Y_hat_df.columns.tolist()
# print(Y_hat_df)
# y_true = Y_hat_df["y"].values.reshape(-1, pred_len, n_series)

INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(val_check_interval=1)` was configured so validation will run after every batch.
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type          | Params
------------------------------------------------
0 | loss          | MQLoss        | 19    
1 | padder_train  | ConstantPad1d | 0     
2 | scaler        | TemporalNorm  | 0     
3 | decomp        | SeriesDecomp  | 0     
4 | linear_trend  | Linear        | 353 K 
5 | linear_season | Linear        | 353 K 
------------------------------------------------
707 K     Trainable params
19        Non-trai

Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  6.30it/s]


/home/user/anaconda3/envs/freqdiff310/lib/python3.10/site-packages/neuralforecast/core.py:199: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [17]:
y_true = Y_hat_df["y"].values.reshape(-1, pred_len, 1)
print(y_true.shape)
y_true[0][:10]

(5069, 192, 1)


array([[-0.61717031],
       [-0.78310749],
       [-0.90936403],
       [-1.13662581],
       [-1.20516508],
       [-1.21057607],
       [-1.32601063],
       [-1.47210749],
       [-1.63083   ],
       [-1.45226717]])